# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [11]:
import numpy as np
#from scipy.misc import imread, imresize
import datetime
import os
import imageio.v2 as imageio # importing imagio for image reading for newer version of scipy
from PIL import Image # Import Image for image resizing

We set the random seed so that the results don't vary drastically.

In [12]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [13]:
train_doc = np.random.permutation(open("C:/Users/kusha/Downloads/Upgrad/Neural_Network/Hand_gesture_recognition_Project/Project_data/train.csv").readlines())
val_doc = np.random.permutation(open("C:/Users/kusha/Downloads/Upgrad/Neural_Network/Hand_gesture_recognition_Project/Project_data/val.csv").readlines())
batch_size = 64

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [14]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =  list (range(30))#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) //batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                #print(f"Full path: {source_path + '/' + t[folder + (batch * batch_size)].split(';')[0]}")
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                #print(imgs)

                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    # Check if width is 160 and crop if necessary
                    if image.shape[1] == 160:  # (Height, Width, Channels)
                        image = image[:120, 20:140]  # Crop to (120, 120)
                    
                    image_resized = np.resize(image, (100, 100, 3))  # Resize to (100, 100)

                    batch_data[folder,idx,:,:,0] = (image_resized[:, :, 0]) / 255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] =  (image_resized[:, :, 1]) / 255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:, :, 2]) / 255.0 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        remaining_samples = len(folder_list) % batch_size
        if remaining_samples > 0:
            batch_data = np.zeros((remaining_samples, len(img_idx), 100, 100, 3))
            batch_labels = np.zeros((remaining_samples, 5))

            for folder in range(remaining_samples):
                folder_name = t[folder + (num_batches * batch_size)].split(';')[0]
                folder_path = os.path.join(source_path, folder_name)
                imgs = sorted(os.listdir(folder_path))

                for idx, item in enumerate(img_idx):
                    if item < len(imgs):
                        image = imageio.imread(os.path.join(folder_path, imgs[item])).astype(np.float32)
                        image_resized = np.resize(image, (100, 100, 3))
                        batch_data[folder, idx, :, :, :] = image_resized / 255.0

                label = int(t[folder + (num_batches * batch_size)].strip().split(';')[2])
                batch_labels[folder, label] = 1

            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [15]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:/Users/kusha/Downloads/Upgrad/Neural_Network/Hand_gesture_recognition_Project/Project_data/train'
val_path = 'C:/Users/kusha/Downloads/Upgrad/Neural_Network/Hand_gesture_recognition_Project/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 5 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 5


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [16]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers import Conv3D, MaxPooling3D, Dropout, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras import optimizers

#write your model here
def build_3d_cnn(input_shape, num_classes):
    model = Sequential()

    # First Conv3D + MaxPooling3D layer
    model.add(Conv3D(16, kernel_size=(3, 3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
    model.add(BatchNormalization())
    
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
    model.add(BatchNormalization())

    # Second Conv3D + MaxPooling3D layer
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
    model.add(BatchNormalization())

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))  # Softmax for classification

    return model

# Example usage
input_shape = (30, 100, 100, 3)  # (time_steps, height, width, channels)
num_classes = 5  # Adjust based on your dataset
model = build_3d_cnn(input_shape, num_classes)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_3 (Conv3D)               │ (None, 30, 100, 100,   │         1,312 │
│                                 │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 15, 50, 50, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 15, 50, 50, 16) │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 15, 50, 50, 32) │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 8, 25, 25, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 25, 25, 32)  │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 8, 25, 25, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_5 (MaxPooling3D)  │ (None, 4, 13, 13, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 4, 13, 13, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │     2,768,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,842,181 (10.84 MB)

 Trainable params: 2,841,957 (10.84 MB)

 Non-trainable params: 224 (896.00 B)

In [17]:
#Conv2D+LSTM model here
# from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, BatchNormalization, Flatten, LSTM, Dropout, Dense

# def build_2d_cnn_lstm(input_shape, num_classes):
#     model = Sequential()

#     # Input shape should be (time_steps, height, width, channels)
#     # First Conv2D + MaxPooling2D layer
#     model.add(TimeDistributed(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'), input_shape=input_shape))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

#     # Second Conv2D + MaxPooling2D layer
#     model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

#     # Third Conv2D + MaxPooling2D layer
#     model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

#     # Fourth Conv2D + MaxPooling2D layer
#     model.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

#     # Fifth Conv2D + MaxPooling2D layer
#     model.add(TimeDistributed(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

#     # Flatten the output of the Conv2D layers
#     model.add(TimeDistributed(Flatten()))

#     # LSTM layer
#     model.add(LSTM(256, return_sequences=False))
#     model.add(Dropout(0.2))

#     # Fully connected layers
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.2))

#     # Output layer for classification
#     model.add(Dense(num_classes, activation='softmax'))

#     return model


# # Example usage
# time_steps = 30
# height, width, channels = 100, 100, 3
# input_shape = (time_steps, height, width, channels)  # (time_steps, height, width, channels)
# num_classes = 5  # Adjust based on your dataset

# model = build_2d_cnn_lstm(input_shape, num_classes)
# model.summary()

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [18]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
learning_rate = 0.001
optimiser = Adam(learning_rate = learning_rate) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_3 (Conv3D)               │ (None, 30, 100, 100,   │         1,312 │
│                                 │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 15, 50, 50, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 15, 50, 50, 16) │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 15, 50, 50, 32) │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 8, 25, 25, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 25, 25, 32)  │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 8, 25, 25, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_5 (MaxPooling3D)  │ (None, 4, 13, 13, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 4, 13, 13, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │     2,768,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,842,181 (10.84 MB)

 Trainable params: 2,841,957 (10.84 MB)

 Non-trainable params: 224 (896.00 B)

None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [19]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [20]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.weights.h5'
print(filepath)

# model_path = model_name + 'model_epoch_{epoch:05d}.h5'
# model.save(model_path)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True, mode='auto', save_freq= 'epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

model_init_2025-01-1421_30_02.618270/model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.weights.h5


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [21]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [22]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  C:/Users/kusha/Downloads/Upgrad/Neural_Network/Hand_gesture_recognition_Project/Project_data/train ; batch size = 64
Epoch 1/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - categorical_accuracy: 0.2070 - loss: 6.3421 Source path =  C:/Users/kusha/Downloads/Upgrad/Neural_Network/Hand_gesture_recognition_Project/Project_data/val ; batch size = 64

Epoch 1: saving model to model_init_2025-01-1421_30_02.618270/model-00001-5.99031-0.22624-1.92787-0.24000.weights.h5
11/11 ━━━━━━━━━━━━━━━━━━━━ 358s 35s/step - categorical_accuracy: 0.2086 - loss: 6.3128 - val_categorical_accuracy: 0.2400 - val_loss: 1.9279 - learning_rate: 0.0010
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - categorical_accuracy: 0.2709 - loss: 2.8744 
Epoch 2: saving model to model_init_2025-01-1421_30_02.618270/model-00002-2.23927-0.26395-1.65565-0.21000.weights.h5
11/11 ━━━━━━━━━━━━━━━━━━━━ 170s 17s/step - categorical_accuracy: 0.2703 - loss: 2.8215 - val_categorical_accuracy: 0.2100 - val_loss: 1.6557 - lear

### Observations
##### - batch size is not impacting training time
##### - Reduce parameters, with padding in Conv3D layers
- Model-1
    ##### conv3D, Batch size: 32, epoc:2, categorical accuracy: 0.2067, loss:2.6 without maxsoft having padding = 'same'
- Model-2
    ##### conv3D, Batch size: 32,  epoc:2, categorical_accuracy: 0.2051 - loss: 1.9303 - val_categorical_accuracy: 0.3200 - val_loss: 1.8485 - learning_rate: 0.001
- Model-3
    ##### conv3D, Batch size: 64,  epoc:2, categorical_accuracy: 0.1875 - loss: 1.8323 - val_categorical_accuracy: 0.2400 - val_loss: 1.6429 - learning_rate: 0.001
- Model-4
    ##### conv3D, Batch size: 64,  epoc:5, categorical_accuracy: 0.2168 - loss: 1.6189 - val_categorical_accuracy: 0.2100 - val_loss: 1.7752 - learning_rate: 0.001
- Model-5
    ##### conv3D, Batch size: 32,  epoc:2, categorical_accuracy: 0.1759 - loss: 23.7709 - val_categorical_accuracy: 0.2200 - val_loss: 1.6074 - learning_rate: 0.01